In [6]:
import pyvista as pv
pv.set_jupyter_backend('ipygany')
import skimage
import trimesh
import torch
import tqdm
import time
import kaolin as kal
from t4dt.metrics import hausdorff, MSDM2
import sys
sys.path.append('..')
from t4dt.t4dt import get_qtt_frame, qtt2tensor3d
from t4dt.utils import sdf2mesh

In [7]:
SDF_PATH = '/scratch2/data/cape_release/meshes/00032/longshort_flying_eagle/posed/sdf_watertight_longshort_flying_eagle.000001.pt'
CKPT_PATH = '/scratch2/projects/t4dt/logs/sweeps/qtt_00032_longshort_flying_eagle_high_ranks_high_thr.pt'
COORDS_PATH = '/scratch2/data/cape_release/meshes/00032/longshort_flying_eagle/coords.pt'

In [8]:
ckpt = torch.load(CKPT_PATH)

In [9]:
scene = ckpt[(-0.05,0.05)][400]['compressed_scene']
lrs = scene.clone()
lrs.round_tt(eps=1e-4)
lrs.numcoef() / lrs.numel()

tensor(0.0041, dtype=torch.float64)

In [16]:
sdf = torch.load(SDF_PATH)['sdf']

In [10]:
frame0_qtt = get_qtt_frame(lrs, 0)

In [11]:
frame0 = qtt2tensor3d(frame0_qtt.torch())

In [12]:
coords = torch.tensor(torch.load(COORDS_PATH)['coords'])

In [13]:
frame0[frame0 < 0] = -1.
frame0[frame0 > 0] = 1.

In [15]:
tmesh = sdf2mesh(frame0, coords)
mesh = pv.wrap(tmesh)
mesh.plot(color='lightgrey', pbr=True, metallic=0.2,
          jupyter_backend='pythreejs')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [33]:
frame_pred = frame0
res = torch.tensor(frame0.shape)
num_sample_points = 30000
tqdm.tqdm.write('Marching cube started')
t0 = time.time()
mesh_pred = sdf2mesh(frame_pred, coords)
tqdm.tqdm.write(f'Marching cube finished. Took: {time.time() - t0} s.')

tqdm.tqdm.write('Marching cube started')
t0 = time.time()
mesh_gt = sdf2mesh(sdf, coords)
tqdm.tqdm.write(f'Marching cube finished. Took: {time.time() - t0} s.')

tqdm.tqdm.write('Sampling points started')
t0 = time.time()
points_gt, _ = trimesh.sample.sample_surface(mesh_gt, num_sample_points)
points_pred, _ = trimesh.sample.sample_surface(mesh_pred, num_sample_points)
tqdm.tqdm.write(f'Sampling points finished. Took: {time.time() - t0} s.')

points_gt = torch.tensor(points_gt[None]).cuda()
points_pred = torch.tensor(points_pred[None]).cuda()

chamfer_distance_error = kal.metrics.pointcloud.chamfer_distance(points_gt, points_pred)[0].detach().cpu()
del points_gt
del points_pred

l2_error = torch.norm(frame_pred - sdf)

tqdm.tqdm.write('Voxelgrid conversion started')
t0 = time.time()
vg_pred = kal.ops.conversions.trianglemeshes_to_voxelgrids(
    torch.tensor(mesh_pred.vertices[None]),
    torch.tensor(mesh_pred.faces),
    res.max().item())
vg_gt = kal.ops.conversions.trianglemeshes_to_voxelgrids(
    torch.tensor(mesh_gt.vertices[None]),
    torch.tensor(mesh_gt.faces),
    res.max().item())
tqdm.tqdm.write(f'Voxelgrid conversion finished. Took: {time.time() - t0} s.')

IoU = kal.metrics.voxelgrid.iou(vg_pred, vg_gt)
del vg_pred
del vg_gt

tqdm.tqdm.write('hausdorff computation started')
t0 = time.time()
hausdorff_dist = hausdorff(
    torch.tensor(mesh_gt.vertices), torch.tensor(mesh_gt.faces),
    torch.tensor(mesh_pred.vertices), torch.tensor(mesh_pred.faces))
tqdm.tqdm.write(f'hausdorff computation finished. Took: {time.time() - t0} s.')

tqdm.tqdm.write('MSDM2 computation started')
t0 = time.time()
MSDM2_err = MSDM2(
    torch.tensor(mesh_gt.vertices), torch.tensor(mesh_gt.faces),
    torch.tensor(mesh_pred.vertices), torch.tensor(mesh_pred.faces))
tqdm.tqdm.write(f'MSDM2 computation finished. Took: {time.time() - t0} s.')
result = {
    'l2': l2_error,
    'chamfer_distance': chamfer_distance_error,
    'IoU': IoU[0],
    'hausdorff': hausdorff_dist,
    'MSDM2': MSDM2_err}

hausdorff computation finished. Took: 90.08560538291931 s.
MSDM2 computation started
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
MSDM2 computation finished. Took: 244.0522162914276 s.


In [34]:
result

{'l2': tensor(6423.2939),
 'chamfer_distance': tensor(4.3948e-05, dtype=torch.float64),
 'IoU': tensor(0.4498),
 'hausdorff': 0.01501531680643125,
 'MSDM2': 0.6110685257034374}

In [35]:
lrs.numcoef() / lrs.numel()

tensor(0.0041, dtype=torch.float64)